# One-Class SVM SimpleEx.
User guide, 2.7. Novelty and Outlier Detection: https://scikit-learn.org/stable/modules/outlier_detection.html<br>
人工データを用いて，One-Class SVMを用いた外れ値（outlier）の検出の仕方を学ぶ。

In [ ]:
import numpy as np
import scipy.stats as stat
from sklearn import svm

import matplotlib.pyplot as plt
%matplotlib inline

## 人工データ
データは正規分布，外れ値は一様分布を用いて生成する。これは，便宜上の割り当てである。<br>
添え字のoutはoutlierの略語。<br>
データは二つのクラスタとし，それぞれ，(0,0),(2,2)を中心とする。 

In [ ]:
np.random.seed(seed=123)
N, Nout = 500, 50
std = 0.2
X1 = stat.norm.rvs(loc=0, scale=std, size=(N,2))
X2 = stat.norm.rvs(loc=2, scale=std, size=(N,2))
Xout = stat.uniform.rvs(loc=-2, scale=6.0, size=(Nout,2))

fig = plt.subplots(figsize=(5,5))
plt.scatter(X1[:,0], X1[:,1], c='w', edgecolors='k', s=30)
plt.scatter(X2[:,0], X2[:,1], c='w', edgecolors='k', s=30)
plt.scatter(Xout[:,0], Xout[:,1], c='r', label='outlier', s=30)
plt.xlim(-3,5)
plt.ylim(-3,5)
plt.grid()
plt.legend(loc='upper left', fontsize=13)
#plt.savefig('PR_OneClassSVM01.pdf')
plt.show()

#### スクリプトの説明
- svm.OneClassSVMの引数，デフォルト値： kernel='rbf', gamma='scale', nu=0.5 
- OC-SVMの計算結果はpredに返る。pred=1：データ、pred=-1：外れ値
- np.whereはタプル型（tuple）で返すことに留意
- sklearn.svm.OneClassSVM:https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html<br>

In [ ]:
X = np.r_[X1, X2, Xout]
clf = svm.OneClassSVM(kernel='rbf', gamma=0.5, nu=0.05)
clf.fit(X)
pred = clf.predict(X)
index = np.where(pred < 0)
n_outlier = index[0].shape[0]
print( type(index), n_outlier)

#### 外れ値の割合
- 人工データの外れ値の割合と検出結果の割合を見る。
- 計算で用いたnuは，外れ値の割合の上限を与える。これと検出結果の割合を見る。1個だけ多く検出したことがわかる。

In [ ]:
print('total=',(2*N+Nout),'  ratio=',(Nout/(2*N+Nout)))
print('outlier=',n_outlier, ' ratio=',n_outlier/(2*N+Nout))

#### 決定境界（decision boundary）
- clf.decision_functio()は，超平面で決定境界を定めているため，元の空間でそのレベルを比較することは難しい。
- しかし，今回は，データか外れ値の領域だけを見ればよいので，データのクラスタを囲んでいる線だけを見ればよい。
- 上図と下図を比較すると，データを外れ値，外れ値をデータに誤検出している例がある。
- しかし，外れ値の定義が数学的に厳密に定められているものではなく，ヒューリスティックに決められることもあるため，この誤検出が直ちに精度が悪いとは言えない部分もある。

In [ ]:
xx, yy = np.meshgrid(np.linspace(-3, 5, 500), np.linspace(-3, 5, 500))
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

In [ ]:
fig = plt.subplots(figsize=(5,5))
plt.contour(xx, yy, Z, colors='k')

plt.scatter(X[:,0], X[:,1], c='w', edgecolors='k', s=30)
plt.scatter(X[index, 0], X[index, 1], c='r', label='outlier', s=30)
plt.xlim(-3,5)
plt.ylim(-3,5)
plt.grid()
plt.legend(loc='upper left', fontsize=13)
#plt.savefig('PR_OneClassSVM02.pdf')
plt.show()

#### 等高線のラベル表示
Contour Label Demo: https://matplotlib.org/stable/gallery/images_contours_and_fields/contour_label_demo.html<br>
matplotlib.pyplot.clabe: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.clabel.html

In [ ]:
fig = plt.subplots(figsize=(5,5))
CS = plt.contour(xx, yy, Z, colors='k')
plt.clabel(CS, fontsize=10, inline=True)

plt.scatter(X[:,0], X[:,1], c='w', edgecolors='k', s=30)
plt.scatter(X[index, 0], X[index, 1], c='r', label='outlier', s=30)
plt.xlim(-3,5)
plt.ylim(-3,5)
plt.grid()
plt.legend(loc='upper left', fontsize=13)
#plt.savefig('PR_OneClassSVM03.pdf')
plt.show()

In [ ]:
Z